## DataLoader Class
The DataLoader Class loads data into a SQLite database from files. All files in the folder should be in the same format and they should be csv files or zipped csv files or any other file type in csv format

In [1]:
# Import libraries
import pandas as pd
import os
from sqlalchemy import create_engine
from loaders import DataLoader

## Parameters that could be changed

In [2]:
DIRECTORY = 'data' # Folder in which files are saved
DBNAME = 'data.sqlite3' # Name of the database
TABLENAME = 'eod' # Name of the table
UNIVERSE = 'NIFTY50'
STOP_LOSS = 4
NUM_STOCKS = 5
CAPITAL = 20000
LEVERAGE = 1
REF_PRICE = 'close'
COMMISSION = 0 # In percent

Don't change the below cell; Just run it.

This updates data of all the files in the folder. If you add a file, just rerun it again so that the database is updated.

In [3]:
connection_string = 'sqlite:///' + DBNAME
engine = create_engine(connection_string)
dl = DataLoader(DIRECTORY, mode='SQL', engine=engine, tablename=TABLENAME)
dl.load_data(parse_dates=['TIMESTAMP'], usecols=range(13)) # This actually loads data

In [4]:
# Now read the entire table back
df = pd.read_sql_table(TABLENAME, engine)

# Check the number of rows; would increase as you put more files in your folder
len(df)

49510

Don't change any cells below.
Just change the **PARAMETERS** above

Of course, you could always *hack the below code*

Backtesting
------------
1. Filter data only for our stock universe
2. Calculate daily returns for each of the stock
3. Carry the returns figure to the next day so that we can view them the next day
4. Group by each day, and apply our strategy
5. Append all the data
6. Calculate prices and performance
7. Evaluate metrics

In [5]:
symbols = pd.read_excel('universe.xlsx', sheet_name=UNIVERSE, header=None).values.ravel()

df = df[df['symbol'].isin(symbols)]
df = df[df['series'] == "EQ"].reset_index(drop=True)
df['ret'] = (df['close']/df['prevclose']) - 1
df = df.sort_values(by='timestamp')
df['ret'] = df.groupby('symbol')['ret'].transform(lambda x: x.shift(1))
grouped = df.groupby('timestamp')
collect = []
for name, group in grouped:
    temp = group.sort_values(by='ret', ascending=False).iloc[:NUM_STOCKS]
    collect.append(temp)
orders = pd.concat(collect)
    

In [6]:
def isPrice(price, high, low):
    if price >= low and price <=high:
        return True
    else:
        False

In [7]:
trading_capital = CAPITAL * LEVERAGE
orders['price'] = orders['open']
orders['stop_loss'] = (orders['price'] * (1+STOP_LOSS*0.01)).round(2)
orders['qty'] = (trading_capital/NUM_STOCKS/orders['price']).round()
orders['sell'] = [price if isPrice(price, high, low) else close for
                 price, high, low, close in 
                 zip(orders['price'], orders['high'], orders['low'], orders[REF_PRICE])]
orders['buy'] = [price if isPrice(price, high, low) else close for
                 price, high, low, close in 
                 zip(orders['stop_loss'], orders['high'], orders['low'], orders[REF_PRICE])]
orders['profit_per_unit'] = orders['sell'] - orders['buy']
orders['total_profit'] = orders['profit_per_unit'] * orders['qty']
cols = ['timestamp', 'symbol', 'buy', 'sell', 'profit_per_unit', 'total_profit']


## Metrics

In [8]:
orders.tail()[['symbol', 'open', 'high', 'low', 'close', 
               'qty', 'buy', 'sell', 'total_profit']]

,symbol,open,high,low,close,qty,buy,sell,total_profit
1221,IBULHSGFIN,1222.00,1222.00,1187.50,1200.85,3.0,1200.85,1222.00,63.45
1207,BPCL,353.05,367.25,348.10,364.00,11.0,367.17,353.05,-155.32
1246,VEDL,235.45,237.75,232.15,234.90,17.0,234.90,235.45,9.35
1219,HINDPETRO,250.10,260.45,248.00,258.35,16.0,260.10,250.10,-160.00
1245,UPL,734.90,735.95,722.00,723.50,5.0,723.50,734.90,57.00


In [9]:
total_profit = orders['total_profit'].sum()
'Returns for the period = {:.2f}%'.format((total_profit/CAPITAL)*100)

'Returns for the period = 3.11%'

In [14]:
filtered = orders[orders['open'] == orders['high']]
filtered.groupby('timestamp').total_profit.sum()

timestamp
2018-08-13    372.90
2018-08-21     37.35
2018-08-23     90.00
2018-08-27     46.40
2018-08-30     67.15
2018-09-11      0.00
2018-09-12     37.50
2018-09-17     63.45
Name: total_profit, dtype: float64

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
by_day = orders.groupby('timestamp').total_profit.sum()
(by_day.cumsum() + CAPITAL).plot(title = 'Portfolio by day')
plt.xlabel('Date')
plt.ylabel('Capital')
plt.show()

<Figure size 640x480 with 1 Axes>

In [12]:
orders.corr()

,open,high,low,close,last,prevclose,tottrdqty,tottrdval,totaltrades,ret,price,stop_loss,qty,sell,buy,profit_per_unit,total_profit
open,1.000000,0.999989,0.999985,0.999974,0.999976,0.999999,-0.148407,-0.084277,-0.174985,0.106600,1.000000,1.000000,-0.341461,1.000000,0.999973,0.940206,-0.015494
high,0.999989,1.000000,0.999991,0.999993,0.999994,0.999987,-0.148559,-0.083319,-0.174466,0.106431,0.999989,0.999989,-0.343225,0.999989,0.999993,0.938756,-0.017994
low,0.999985,0.999991,1.000000,0.999994,0.999994,0.999984,-0.149443,-0.084238,-0.175400,0.105659,0.999985,0.999985,-0.343661,0.999985,0.999993,0.938542,-0.017418
close,0.999974,0.999993,0.999994,1.000000,1.000000,0.999973,-0.149469,-0.083687,-0.175015,0.105847,0.999974,0.999974,-0.344469,0.999974,1.000000,0.937734,-0.019398
last,0.999976,0.999994,0.999994,1.000000,1.000000,0.999975,-0.149456,-0.083690,-0.174994,0.105973,0.999976,0.999976,-0.344395,0.999976,1.000000,0.937823,-0.019328
prevclose,0.999999,0.999987,0.999984,0.999973,0.999975,1.000000,-0.148386,-0.084348,-0.175093,0.106624,0.999999,0.999999,-0.341307,0.999999,0.999972,0.940172,-0.015462
tottrdqty,-0.148407,-0.148559,-0.149443,-0.149469,-0.149456,-0.148386,1.000000,0.882100,0.862401,0.077322,-0.148407,-0.148407,0.259719,-0.148407,-0.149516,-0.088013,0.014557
tottrdval,-0.084277,-0.083319,-0.084238,-0.083687,-0.083690,-0.084348,0.882100,1.000000,0.942020,0.103981,-0.084277,-0.084277,-0.081493,-0.084277,-0.083722,-0.104828,-0.094205
totaltrades,-0.174985,-0.174466,-0.175400,-0.175015,-0.174994,-0.175093,0.862401,0.942020,1.000000,0.121045,-0.174985,-0.174985,0.021897,-0.174985,-0.175068,-0.160480,-0.102648
ret,0.106600,0.106431,0.105659,0.105847,0.105973,0.106624,0.077322,0.103981,0.121045,1.000000,0.106600,0.106600,-0.021232,0.106600,0.105952,0.130110,-0.037809
